In [1]:
import torch
import torch.nn as tnn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
from tensorboardX import SummaryWriter

from collections import deque
import os
import copy
import numpy as np
from sklearn.manifold import Isomap
from sklearn.neighbors import NearestNeighbors
from locally_linear import LocallyLinearBackward

from tqdm import tqdm
from collections import deque
import os

from tensorboardX import SummaryWriter
from model.vgg_tiny import Conv, Fc

In [2]:
os.system('nvidia-smi -q -d Memory |grep -A4 GPU|grep Free >tmp')
memory_gpu=[int(x.split()[2]) for x in open('tmp','r').readlines()]
os.environ['CUDA_VISIBLE_DEVICES']=str(np.argmax(memory_gpu))
os.system('rm tmp')

0

In [3]:
BATCH_SIZE = 50
LEARNING_RATE = 0.01
EPOCH = 100
n_dimentions = 7
isomap_number = 1000
write_log = True
n_neighbors = 5

embedding = Isomap(n_components=n_dimentions, n_neighbors=n_neighbors)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
def isomap(feature_queue, n_components):
    length = len(feature_queue)
    for i in range(length):
        if i == 0:
            feature_tmp = feature_queue.popleft()
            features = feature_tmp
            feature_to_use = feature_tmp
        else:
            feature_tmp = feature_queue.popleft()
            features = np.concatenate((features, feature_tmp), axis=0)
    
#     np.save('x.npy', features)
#     np.save('x_50.npy', feature_to_use)
    feature_input = features
#     embedding = Isomap(n_components=n_components)
    transformed = embedding.fit_transform(feature_input)
    
#     np.save('y.npy', transformed)
    return transformed, feature_to_use

In [5]:
def isomap_back(X_Que,Y_use,Y_all,Error):
#Error has dim:p*N, where p is the dims of every object after isomap N is the batchsize
#Y is the feature during the forward process
#X is the feature before Isomap

    k=4
    E=Error.cpu().numpy()
    Y_use = Y_use.detach().cpu().numpy()
    length=len(X_Que)
    for i in range(length):
        if i == 0:
            feature_tmp = X_Que.popleft()
            X_use=feature_tmp
            X = feature_tmp
        else:
            feature_tmp = X_Que.popleft()
            X = np.concatenate((X, feature_tmp), axis=0)
#     np.save('Y_50.npy', Y_use)
    Yb=Y_use+E
#     np.save('Y_hat_50.npy', Yb)
#Calculate all the distances between Yb and all Y
    n=Yb.shape
    total=Y_all.shape
    for i in range(n[0]):
        dis=np.zeros(total[0])
        Yi=Yb[i]
        for j in range(total[0]):
            z=(Yi-Y_all[j]).reshape(-1,1)
            dis[j]=np.matmul(z.transpose(),z)
        idx = np.argpartition(dis, k)[0:k]
        for m in range(k):
            if m==0:
                Y_near=Y_all[idx[m]].reshape([1,-1])
                X_near = X[idx[m]].reshape([1,-1])
                Y_bar=Yi.reshape([1,-1])
            else:
                Y_near=np.concatenate((Y_near,Y_all[idx[m]].reshape([1,-1])),axis=0)
                X_near=np.concatenate((X_near,X[idx[m]].reshape([1,-1])),axis=0)
                Y_bar=np.concatenate((Y_bar,Yi.reshape([1,-1])),axis=0)
        Y_all_hat = np.concatenate((Yb, Y_all[BATCH_SIZE:]), axis=0)
#         np.save('Y_all_hat.npy', Y_all_hat)
        tmp=np.mat(Y_bar-Y_near)
        Z=tmp*tmp.transpose()
        One=np.mat(np.ones([k, 1]))
        X_near=np.mat(X_near)
        w=(np.linalg.pinv(Z))*One/(One.transpose()*(np.linalg.pinv(Z))*One)
        if i==0:
            X_back=(X_near.transpose()*w).reshape([1,-1])
        else:
            X_back=np.concatenate((X_back,(X_near.transpose()*w).reshape([1,-1])),axis=0)
    return X_back

In [6]:
def train(epoch):
    vgg_conv.train()
    vgg_fc.train()
    all_correct = 0
    total = 0
    train_img_queue = deque(maxlen=isomap_number//BATCH_SIZE)    #构建输入图像的队列
    train_label_queue = deque(maxlen=isomap_number//BATCH_SIZE) #构建label的队列
    train_vec_queue = deque(maxlen=isomap_number//BATCH_SIZE)    #构建卷积网络输出向量的队列
    for batch_idx, (images, labels) in enumerate(tqdm(trainLoader)):
        total += images.shape[0]
        train_img_queue.append(images)   #入队是append，出队是popleft
        train_label_queue.append(labels)

        # Forward + Backward + Optimize

    #     optimizer1.zero_grad()
    #     optimizer2.zero_grad()

        outputs1 = vgg_conv(images.to(device)) #卷积网络的输出，将图片embedding成512维向量，the shape of output is (batch_size, 512)
        #print(images, outputs1)
        train_vec_queue.append(outputs1.detach().cpu().numpy())
    #     print(train_vec_queue.qsize())
    #     print(train_vec_queue.get_nowait().shape)

        if len(train_img_queue) == isomap_number//BATCH_SIZE:  #等队列满了之后，开始让所有图片进入isomap，然后pop出队首的数据进行反向传播
            isomap_forward, feature_to_use = isomap(copy.deepcopy(train_vec_queue), n_components=32)#将1000张图片通过卷积层得到的embedding向量输入isomap层，获得降维后的结果
            #isomap: numpy.ndarray   feature_to_use: numpy.ndarray

            train_label_tmp = copy.deepcopy(train_label_queue)
            for i in range(len(train_label_tmp)):
                if i == 0:
                    feature_tmp = train_label_tmp.popleft()
                    label_npy = feature_tmp
                else:
                    feature_tmp = train_label_tmp.popleft()
                    label_npy = np.concatenate((label_npy, feature_tmp), axis=0)
    #         np.save('isomap_label.npy', label_npy)

            feature_to_use = torch.Tensor(feature_to_use)
            if outputs1.is_cuda:
                batch_feature = torch.from_numpy(isomap_forward[:BATCH_SIZE]).cuda()
            else:
                batch_feature = torch.from_numpy(isomap_forward[:BATCH_SIZE])

            batch_feature = batch_feature.float()

    #         img_tmp = train_img_queue.popleft()
    #         label_tmp = train_label_queue.popleft()
    #         vec_tmp = train_vec_queue.popleft()

            isomap_feature = batch_feature
            isomap_feature.requires_grad = True
            outputs2 = vgg_fc(isomap_feature)
            batch_label = train_label_queue.popleft()
            loss2 = cost2(outputs2, batch_label.squeeze().to(device))

            #---------------------------------------
            optimizer2.zero_grad()
            loss2.backward()
            optimizer2.step()

            grad = isomap_feature.grad

            E = grad*LEARNING_RATE
            x_hat = isomap_back(copy.deepcopy(train_vec_queue), isomap_feature, isomap_forward, E)
    #         np.save('x_hat_50.npy', x_hat)
            #all X:copy.deepcopy(train_vec_queue) Y:isomap_feature    all Y: isomap_forward     y error:E
            x_hat = torch.Tensor(x_hat)
            loss1 = cost1(feature_to_use.to(device), x_hat.to(device))
            loss1.requires_grad = True
            optimizer1.zero_grad()
            loss1.backward()
            optimizer1.step()

            train_vec_queue.popleft()
            train_img_queue.popleft()

            pred = torch.max(outputs2.data, 1)[1]
            train_correct = (pred == batch_label.to(device)).sum()
            all_correct += train_correct

            print('epoch:{}/{}  batch:{}/{}  loss1:{:.6f}  loss2:{:.6f}  acc:{:.4f} all_acc:{:.4f}'.format(epoch, EPOCH, batch_idx,
                                                                             data_train.__len__() // BATCH_SIZE, loss1,
                                                                             loss2, float(train_correct) / BATCH_SIZE,
                                                                             float(all_correct) / total))

In [7]:
def test(epoch):
    vgg_conv.eval()
    vgg_fc.eval()
    features_number = 0
    correct = 0
    total = 0
    total_loss = 0
    for batch_idx, (data, target) in enumerate(trainLoader):
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()

        features = vgg_conv(data)
        if features_number == 0:
            features_tensor = features.cpu().detach()
            target_tensor = target
            features_number += BATCH_SIZE
        elif features_number < isomap_number - BATCH_SIZE:
            features_tensor = torch.cat((features_tensor, features.cpu().detach()), dim=0)
            target_tensor = torch.cat((target_tensor, target), dim=0)
            features_number += BATCH_SIZE
        else:
            break

    for batch_idx, (data, target) in enumerate(testLoader):
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()

        features = vgg_conv(data)
        features_array = torch.cat((features_tensor[:isomap_number - BATCH_SIZE], features.cpu().detach()),
                                   dim=0).numpy()
        target_tensor = target
        if torch.cuda.is_available():
            transformed = torch.Tensor(embedding.transform(features_array)[isomap_number - BATCH_SIZE:]).cuda()
        else:
            transformed = torch.Tensor(embedding.transform(features_array)[isomap_number - BATCH_SIZE:])
        output = vgg_fc(transformed)

        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target_tensor.data.view_as(pred)).cpu().sum()
        total += target_tensor.shape[0]

        loss = cost2(output, target_tensor.squeeze().to(device))
        total_loss += loss.item()

    log = ('Test Epoch: {}\tAcc:[{}/{}]{:.6f} Loss: {:.6f}'.format(
                epoch, int(correct), int(total), float(correct)/float(total)*100, total_loss))
    print(log)
    if write_log:
        writer.add_scalar('Isomap_test_loss', total_loss, epoch)
        writer.add_scalar('Isomap_test_accuracy', float(correct) / float(total) * 100, epoch)


In [8]:
transform = transforms.Compose([
    transforms.ToTensor()])

data_train = dsets.MNIST(root = "./data/",
                         transform=transform,
                            train = True,
                            download = True)

data_test = dsets.MNIST(root="./data/",
                        transform=transform,
                           train = False)

trainLoader = torch.utils.data.DataLoader(dataset=data_train, batch_size=BATCH_SIZE, shuffle=True)
testLoader = torch.utils.data.DataLoader(dataset=data_test, batch_size=BATCH_SIZE, shuffle=False)

In [9]:
vgg_conv = Conv().to(device)
vgg_fc = Fc(input_channel=7, output_channel=10).to(device)

vgg_conv.load_state_dict(torch.load('./model_saved/conv_train_isomap_pureconv_SGD_downdimension_50_isomap_number_5000_neighbors_7_epoch_19.pkl'))

/DATA4_DB3/data/wen/isomap/workspace/model/vgg_tiny.py:85: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  m.weight = nn.init.xavier_normal(m.weight)


In [10]:
isomap_feature = torch.empty(BATCH_SIZE, n_dimentions, requires_grad=True, device=device)
cost1 = tnn.MSELoss()
cost2 = tnn.CrossEntropyLoss()
optimizer1 = torch.optim.SGD(vgg_conv.parameters(), lr=LEARNING_RATE)
optimizer2 = torch.optim.SGD(vgg_fc.parameters(), lr=LEARNING_RATE)

In [ ]:
file_name = 'train_new_isomap_pureconv_stableisomap_downdimension_{}_isomap_number_{}_neighbors_{}_batch_{}'.format(
        n_dimentions, isomap_number, n_neighbors, BATCH_SIZE)
if write_log:
        writer = SummaryWriter('./runs/{}'.format(file_name))
        if not os.path.exists('./model_saved/{}'.format(file_name)):
            os.makedirs('./model_saved/{}'.format(file_name))
else:
        writer = None

# Train the model
for epoch in range(EPOCH):
    train(epoch)
    test(epoch)
    

  2%|▏         | 20/1200 [00:09<11:33,  1.70it/s]

epoch:0/100  batch:19/1200  loss1:5.632685  loss2:2.712877  acc:0.0000 all_acc:0.0000
epoch:0/100  batch:20/1200  loss1:7.875229  loss2:2.293536  acc:0.1400 all_acc:0.0067


  2%|▏         | 22/1200 [00:25<55:39,  2.84s/it]

epoch:0/100  batch:21/1200  loss1:5.010005  loss2:2.410365  acc:0.1400 all_acc:0.0127


  2%|▏         | 23/1200 [00:33<1:23:28,  4.25s/it]

...

epoch:0/100  batch:27/1200  loss1:7.332511  loss2:2.321125  acc:0.0400 all_acc:0.0286


  2%|▏         | 29/1200 [01:19<2:19:14,  7.13s/it]

epoch:0/100  batch:28/1200  loss1:13.570882  loss2:2.440329  acc:0.1400 all_acc:0.0324


  2%|▎         | 30/1200 [01:25<2:13:17,  6.84s/it]

epoch:0/100  batch:29/1200  loss1:4.275620  loss2:2.336364  acc:0.1000 all_acc:0.0347
